In [1]:
# dependancies
# possibly more than needed
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import chardet

In [2]:
# Make list of column/headers for Great Big Dataframe (GBDf) with empty cells - Paul

# FIPS
# (five-digit code: the first two represent state, the last three represent county in that state)

# County
# (name, e.g. Jackson County in many states, but also Orleans Parish of Louisiana, or the Aleutians East Borough of Alaska)

# State (name)

# Pop
# (2019 county population estimate)

# PopDens
# (Population density; derived from county 2019 population estimate divided by county land mass in another dataframe)

# MskScore
# (Mask-wearing score derived from pct in each county who said never, rarely sometimes, frequently, always wear a mask when going out)

# CaseRate
# (cumulative COVID cases per 100,000 population by county January 21–July 14, 2020)

# DeathRate
# (cumulative COVID deaths per 100,000 population by county January 21–July 14, 2020)

# Income
# (Mean per-capita income by county; this is a bonus-round question)

In [18]:
columnNames = ["FIPS", "County", "State", "Pop", "PopDens", "MskScore", "CaseRate", "DeathRate", "Income"]
GBDf = pd.DataFrame(columns = columnNames)
# placeholderData = ["01001", "Autauga", "Alabama", "55869", "94.0", "7.51", "0", "1335", "32"]
placeholderData = {"FIPS":"01001", "County":"Autauga", "State":"Alabama", "Pop":55869, "PopDens":94.0, "MskScore":7.51, "CaseRate":1335, "DeathRate":32, "Income":0}
GBDf = GBDf.append(placeholderData, ignore_index=True)
GBDf

,FIPS,County,State,Pop,PopDens,MskScore,CaseRate,DeathRate,Income
0,01001,Autauga,Alabama,55869,94.0,7.51,1335,32,0


In [4]:
# Aleena's code starts here

In [5]:
# 1. US Census 2010-2019
censusDataReadMeURL = "https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html"

# This is where the census data .CSV lives locally...:
censusDataFilepath = "Resources/co-est2019-alldata_exp.csv"

print(f"{censusDataFilepath} is {round(os.path.getsize(censusDataFilepath)/1024/1024, 2)} megabytes (MB).\nMore info here:\n{censusDataReadMeURL}")

Resources/co-est2019-alldata_exp.csv is 0.15 megabytes (MB).
More info here:
https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html


In [6]:
censusData = pd.read_csv(censusDataFilepath, encoding="iso-8859-1")
censusData

,FIPS,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,POPESTIMATE2019
0,1000,1,0,Alabama,Alabama,4779736,4903185
1,1001,1,1,Alabama,Autauga County,54571,55869
2,1003,1,3,Alabama,Baldwin County,182265,223234
3,1005,1,5,Alabama,Barbour County,27457,24686
4,1007,1,7,Alabama,Bibb County,22915,22394
...,...,...,...,...,...,...,...
3188,56037,56,37,Wyoming,Sweetwater County,43806,42343
3189,56039,56,39,Wyoming,Teton County,21294,23464
3190,56041,56,41,Wyoming,Uinta County,21118,20226
3191,56043,56,43,Wyoming,Washakie County,8533,7805


In [7]:
# Aleena's code ends here for now

In [8]:
# Matt's code starts here

In [9]:
# 2. COVID-19 cases
caseDataReadMeURL = "https://github.com/nytimes/covid-19-data/blob/master/README.md"

# This is where the .CSV lives locally...:
caseDataFilepath = "Resources/us-counties.csv"

print(f"The file at {caseDataFilepath} is {round(os.path.getsize(caseDataFilepath)/1024/1024, 2)} MB.\nMore info here:\n{caseDataReadMeURL}")

The file at Resources/us-counties.csv is 39.29 MB.
More info here:
https://github.com/nytimes/covid-19-data/blob/master/README.md


In [10]:
caseData = pd.read_csv(caseDataFilepath, encoding = "iso-8859-1")
caseData

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0
...,...,...,...,...,...,...
1011388,2021-02-08,Sweetwater,Wyoming,56037.0,3559,33.0
1011389,2021-02-08,Teton,Wyoming,56039.0,3257,7.0
1011390,2021-02-08,Uinta,Wyoming,56041.0,1988,12.0
1011391,2021-02-08,Washakie,Wyoming,56043.0,869,26.0


In [11]:
# Matt's code ends here for now

In [12]:
# Emerson's code starts here

In [13]:
# 3. mask-wearing survey
maskWearingDataReadMeURL = "https://github.com/nytimes/covid-19-data/blob/master/README.md"
# This is where the census data .CSV lives locally...:
maskWearingDataFilepath = "Resources/mask-use-by-county-exp.csv"

print(f"{maskWearingDataFilepath} is {round(os.path.getsize(maskWearingDataFilepath)/1024/1024, 2)} MB.\nMore info here:\n{maskWearingDataReadMeURL}")

Resources/mask-use-by-county-exp.csv is 0.12 MB.
More info here:
https://github.com/nytimes/covid-19-data/blob/master/README.md


In [14]:
maskWearingData = pd.read_csv(maskWearingDataFilepath)
maskWearingData

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,1001,5.30%,7.40%,13.40%,29.50%,44.40%
1,1003,8.30%,5.90%,9.80%,32.30%,43.60%
2,1005,6.70%,12.10%,12.00%,20.10%,49.10%
3,1007,2.00%,3.40%,9.60%,27.80%,57.20%
4,1009,5.30%,11.40%,18.00%,19.40%,45.90%
...,...,...,...,...,...,...
3137,56037,6.10%,29.50%,23.00%,14.60%,26.80%
3138,56039,9.50%,15.70%,16.00%,24.70%,34.00%
3139,56041,9.80%,27.80%,15.40%,20.70%,26.40%
3140,56043,20.40%,15.50%,6.90%,28.50%,28.70%


In [19]:
scale = 10
divisions = 5
interval = scale / (divisions-1)
print(f"This will use the results of the NYT survey to score each county on a scale from 0 to {scale} where")
print(f"never = 0")
print(f"rarely = {interval}")
print(f"sometimes = {interval*2}")
print(f"frequently = {interval*3}")
print(f"always = {interval*4}")

This will use the results of the NYT survey to score each county on a scale from 0 to 10 where
never = 0
rarely = 2.5
sometimes = 5.0
frequently = 7.5
always = 10.0


In [21]:
# Each county can be represented by a mask-wearing score.
# maskWearingData.insert(1, "Mask Wearing Score", 0)
# maskWearingData

# maskWearingData["Mask Wearing Score"] = maskWearingData["RARELY"] * interval + maskWearingData["RARELY"] * interval * 2 +  maskWearingData["SOMETIMES"] * interval * 3 + maskWearingData["FREQUENTLY"] * interval * 4  + maskWearingData["ALWAYS"] * interval * 5

In [ ]:
# maskWearingScore = rarely*interval + sometimes*interval*2 + frequently*interval*3 + always*interval*4

In [ ]:
# Emerson's code ends here for now

In [ ]:
# Paul's code starts here

In [23]:
# 4. land mass (and thence population density)
landMassDataReadMeURL = ""
# This is where the land mass data .CSV lives locally...:
landMassDataFilepath = "Resources/counties_by_land_area.csv"

print(f"{landMassDataFilepath} is {round(os.path.getsize(landMassDataFilepath)/1024/1024, 2)} MB.\nMore info here:\n{landMassDataReadMeURL}")

Resources/counties_by_land_area.csv is 0.37 MB.
More info here:



In [24]:
landMassData = pd.read_csv(landMassDataFilepath)
landMassData

,Areaname,STCOU,LND010190F,LND010190D,LND010190N1,LND010190N2,LND010200F,LND010200D,LND010200N1,LND010200N2,...,LND110210N1,LND110210N2,LND210190F,LND210190D,LND210190N1,LND210190N2,LND210200F,LND210200D,LND210200N1,LND210200N2
0,UNITED STATES,0,0,3787425.08,0,0,0,3794083.06,0,0,...,0,0,0,251083.35,0,0,0,256644.62,0,0
1,ALABAMA,1000,0,52422.94,0,0,0,52419.02,0,0,...,0,0,0,1672.71,0,0,0,1675.01,0,0
2,"Autauga, AL",1001,0,604.49,0,0,0,604.45,0,0,...,0,0,0,8.48,0,0,0,8.48,0,0
3,"Baldwin, AL",1003,0,2027.08,0,0,0,2026.93,0,0,...,0,0,0,430.55,0,0,0,430.58,0,0
4,"Barbour, AL",1005,0,904.59,0,0,0,904.52,0,0,...,0,0,0,19.59,0,0,0,19.61,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3193,"Sweetwater, WY",56037,0,10491.73,0,0,0,10491.17,0,0,...,0,0,0,65.86,0,0,0,65.87,0,0
3194,"Teton, WY",56039,0,4221.96,0,0,0,4221.80,0,0,...,0,0,0,214.02,0,0,0,214.04,0,0
3195,"Uinta, WY",56041,0,2087.66,0,0,0,2087.56,0,0,...,0,0,0,5.91,0,0,0,5.90,0,0
3196,"Washakie, WY",56043,0,2242.85,0,0,0,2242.75,0,0,...,0,0,0,2.69,0,0,0,2.69,0,0


In [ ]:
# counties_by_land_area.csv